### Day 20: Race Condition

Link: https://adventofcode.com/2024/day/20

To solve this problem, we first solve the maze using breadth-first search to find the path to the goal. While doing that, we also track the least amount of time it takes to reach each position. Since there's only one path to the goal, all valid positions will be covered. For each position in our track, we then generate cheat states where the racer passes through one wall and ends up on a valid path in the maze. We can simplify this part by finding all path positions within a Manhattan distance of two from the current position. It doesn't matter if we crossed walls or not, only the shortest time it takes to reach each point. Finally, we compare the time saved by reaching those positions with the time from our previous tracking record. If it's greater than or equal to 100, we add it to our final answer.

In [1]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
from dataclasses import dataclass
from collections import deque


@dataclass(frozen=True)
class Position:
    row: int
    column: int


@dataclass
class State:
    position: Position
    time: int


grid: list[list[str]] = []


for line in lines:
    grid.append(line.strip())


def find_start_position() -> Position:
    for row in range(len(grid)):
        for column in range(len(grid[0])):
            if grid[row][column] == "S":
                return Position(row=row, column=column)

    raise Exception("Start position not found")


start_position = find_start_position()
to_visit = deque([State(position=start_position, time=0)])
path_track: dict[Position, int] = {}
visited: set[Position] = set()


def is_within_grid(position: Position) -> bool:
    return 0 <= position.row < len(grid) and 0 <= position.column < len(grid[0])


def is_wall(position: Position) -> bool:
    return grid[position.row][position.column] == "#"


def is_end(position: Position) -> bool:
    return grid[position.row][position.column] == "E"


def move(position: Position) -> list[Position]:
    new_positions: list[Position] = []
    actions = [
        (-1, 0),  # Up
        (0, 1),  # Right
        (1, 0),  # Down
        (0, -1),  # Left
    ]

    for add_row, add_column in actions:
        new_positions.append(
            Position(
                row=position.row + add_row,
                column=position.column + add_column,
            )
        )

    return new_positions


while to_visit:
    state = to_visit.popleft()

    if state.position in visited:
        continue

    visited.add(state.position)
    path_track[state.position] = state.time

    if is_end(state.position):
        break

    for new_position in move(state.position):
        if not is_wall(new_position):
            to_visit.append(State(position=new_position, time=state.time + 1))


def generate_cheat_states(
    original_position: Position, original_time: int
) -> list[State]:
    max_manhattan_distance = 2
    cheat_states: list[State] = []

    for row in range(
        original_position.row - max_manhattan_distance,
        original_position.row + max_manhattan_distance + 1,
    ):
        for column in range(
            original_position.column - max_manhattan_distance,
            original_position.column + max_manhattan_distance + 1,
        ):
            manhattan_distance = abs(original_position.row - row) + abs(
                original_position.column - column
            )

            if manhattan_distance > max_manhattan_distance:
                continue

            new_position = Position(row=row, column=column)

            if not is_within_grid(new_position):
                continue

            if is_wall(new_position):
                continue

            cheat_state = State(
                position=new_position, time=original_time + manhattan_distance
            )
            cheat_states.append(cheat_state)

    return cheat_states


min_time_save = 100
cheats_count = 0


for position, time in path_track.items():
    cheat_states = generate_cheat_states(position, time)

    for cheat_state in cheat_states:
        if path_track[cheat_state.position] - cheat_state.time >= min_time_save:
            cheats_count += 1


print(cheats_count)